In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
%matplotlib inline
from sklearn.model_selection import train_test_split
from scipy import optimize
import numpy as np

In [2]:
raw = fetch_california_housing()
X = pd.DataFrame(data=raw['data'], columns=raw['feature_names'])
y = pd.Series(raw['target'])
X
y


0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Length: 20640, dtype: float64

In [3]:
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
scaler = StandardScaler()
X[features] = scaler.fit_transform(X[features])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Define the TensorFlow graph:
* Create placeholders for the input features (X) and target variable (y).
* Create variables for the model's weights (W) and bias (b).
* Define the linear regression model using the equation: y_pred = X * W + b.
* Define the loss function as the mean squared error between the predicted values and the true values.
* Choose an optimizer (e.g., Gradient Descent) to minimize the loss function.

In [4]:
tf.compat.v1.disable_eager_execution()
X_placeholder = tf.compat.v1.placeholder(tf.float64, shape=[None, 8])
y_placeholder = tf.compat.v1.placeholder(tf.float64,shape=[None])

In [5]:

W = tf.Variable(tf.random.normal([X_train.shape[1], 1], dtype=tf.float64))
b = tf.Variable(tf.random.normal([1], dtype=tf.float64))

In [6]:
y_pred= tf.matmul(X_placeholder, W) + b

In [7]:
loss = tf.reduce_mean(tf.square(y_pred - y_placeholder))
loss

<tf.Tensor 'Mean:0' shape=() dtype=float64>

Train the model:

* Initialize TensorFlow session.
* Initialize the model's variables.
* Set the number of training epochs and the learning rate.
* For each epoch, iterate through the training dataset and update the model's parameters using the optimizer.
* Print the training loss at regular intervals.


In [ ]:
optimizer = tf.optimizers.SGD(learning_rate=0.01)

num_epochs = 4
batch_size = 2
total_batches = int(np.ceil(len(X_train) / batch_size))

for epoch in range(num_epochs):
    epoch_loss = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.BinaryAccuracy()

    for batch in range(total_batches):
        start_index = batch * batch_size
        end_index = start_index + batch_size

        x_batch = X_train[start_index:end_index]
        y_batch = y_train[start_index:end_index]

        with tf.GradientTape() as tape:
            y_pred = tf.matmul(x_batch, W) + b
            loss = tf.reduce_mean(tf.square(y_pred - y_batch))

        gradients = tape.gradient(loss, [W, b])
        optimizer.apply_gradients(zip(gradients, [W, b]))

        epoch_loss(loss)
        epoch_accuracy(y_batch, y_pred)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Loss: {epoch_loss.result()}")
    print(f"Accuracy: {epoch_accuracy.result()}")

    epoch_loss.reset_states()
    epoch_accuracy.reset_states()


Epoch 1/4
Loss: Tensor("Identity_33024:0", shape=(), dtype=float32)
Accuracy: Tensor("Identity_33025:0", shape=(), dtype=float32)
Epoch 2/4
Loss: Tensor("Identity_66050:0", shape=(), dtype=float32)
Accuracy: Tensor("Identity_66051:0", shape=(), dtype=float32)
